In [4]:
########################################################################################################################
#Pearl's Message Passing Algorithm

#######################################################################################################################
import numpy as np
from collections import defaultdict

# "ACCEPT NUMBER OF NODES AND NODES" ###################################################################################

file = open("test_fin.txt")
v = int(file.readline())
nodes = []
for i in range(v):
    line = file.readline().split() # read each node
    nodes.append(line[0]) # store all nodes in the list
nodes = list(nodes)  # list "nodes" contains all nodes present in given graph

#"ACCEPT NUMBER OF EDGES AND EDGES"#####################################################################################

e = int(file.readline())
edges = []
for i in range(e):
    edges.append(file.readline().split())  #add edges with "parent child" as format in the text file

#"ACCEPT NODES VALUES"#################################################################################################

node_values = {}
for i in range(v):
    x = file.readline().split()
    node_values[x[0]] = x[1:]      # "node_values" list contains all possible values each node can take

#"BUILD GRAPH / EDGES AND STORE PARENTS AND CHILDREN IN DICTIONARY"###################################################

parents = defaultdict(list)
child = defaultdict(list)
for edge in edges:
    parents[edge[1]].append(edge[0])   # based on edges formed create dictionary of parents [key --> value] as [child->parent]
    child[edge[0]].append(edge[1])  # based on edges formed create of dictionary of child  [key --> value] as [parent->child]

#"Display Graph"######################################################################################################

print ("Directed Acyclic graph")
for node in nodes:
    print("for Node ",  node)
    print("\t we have structure as ",  str(parents[node]))
    print("\t\t\t\t ------> ", node)
    print("\t\t\t\t\t\t------> ",str(child[node]))
    
#"ACCEPT MARGINAL AND CONDITIONAL PROBABLITIES"########################################################################

nmp = int(file.readline())
mp = {}
for i in range(nmp):
    x = file.readline().split()
    mp[x[0]] = float(x[1]) # mp is dictionary where "key" is the node value and its corresponding value is the marginal probablity.

ncp = int(file.readline())
cp = defaultdict(dict)
for i in range(ncp):
    x = file.readline().split()
    cp[x[0]][x[1]] = float(x[2]) # "cp" is dictionary of dictionary Key : {key:values}, where higher level key is node  
                                 # lower level key are node values and its corresponding values are probablities.

#" Function that has list of variables used in the code and that needs to be initialized for each instantiation"########

def variable_set():
    evidence = []   # list containing evidence
    evidence_inst = [] # value that evidence takes
    updated_prob = {}  # Dictionary of updated probablities with node values as its keys,contains updated values for all node values
    lambda_value = {}  # Dictionary of lambda values
    pi_value = {}      # Dictionary of lambda values
    lambda_msg = defaultdict(dict) # lambda message from child to its parents
    pi_msg = defaultdict(dict)     # PI message from parent to its children

    return evidence,evidence_inst,updated_prob,lambda_value,pi_value,lambda_msg,pi_msg

#"Function to Initialize network"#####################################################################################

def initialize_network():
    for X in nodes:  # for each nodes in the graph
        for x in node_values[X]:  # for each values the node takes
            lambda_value[x] = 1  # update lambda values
        if parents[X]:          # for its parents send lambda message with value 1
            for Z in parents[X]:
                for z in node_values[Z]:
                    lambda_msg[X][z] = 1  # initialize lambda message to be 1
        
        if child[X]:           # Similary for all its child send pi message with value to be one
            for Y in child[X]:
                for x in node_values[X]:
                    pi_msg[Y][x] = 1   # initialize pi message to be one
                
    for X in nodes:                # check for presence of root nodes in the graph
        if not parents[X]:         # the condition to check whether that particular node has parent
            for r in node_values[X]:
                pi_value[r] = mp[r] # update pi value of parent to its marginal probablity
                updated_prob[r] = mp[r] # Update its conditional probablity to be as marginal probablity              
            for W in child[X]:
                PI_MSG(X,W)   # To all the children of root nodes send PI message


#"Function to Instantiate / Update network"#########################################################################

def Instantiate_network(V, vcap):
    evidence.append(V)      # update the list of evidence based on call of instantiate function
    evidence_inst.append(vcap)  # Update the value that instiantiated node takes

    for v in node_values[V]: # for each nodes values of instiantiated node
        if v == vcap:       # if instiantiated value matches
            lambda_value[v] = 1 # update lambda value to 1
            pi_value[v] = 1     # update PI value to 1
            updated_prob[v]=1   # update its conditional probablity to be one
        else:                   # Otherwise 
            lambda_value[v] = 0 # update lambda value to 0
            pi_value[v] = 0      # update PI value to 0
            updated_prob[v] = 0 # update its conditional probablity to be zero
         
    for Z in parents[V]:     # For all parents of instiantiated node
        if Z not in evidence: # and if parent is not in evidence then
            LAMBDA_MSG(V,Z)   # send Labmda message from child(instiantiated node) to its parent

    for Y in child[V]:  # For all children of instiantiated node
        PI_MSG(V, Y)    # send PI message from parent(instiantiated node) to its child

#"Function to pass lambda message" #######################################################################################

def LAMBDA_MSG(Y, X):  # Lambda message from child to its parent
    parents_tilda_Y = [] # List that contains the other parent 
    for parent in parents[Y]: # for each parent 
        if parent!=X:   # condition to get other parents
            parents_tilda_Y.append(parent) # update other parents list accordingly
 
    c = Instances(parents_tilda_Y) # this is call for function that returns all possible combination of node values
    zee = dict()
    for x in node_values[X]: # for each node values of X (parent)
        S=0  # S is variable that holds the value sum and it is initialized to zero
        if not c:  # if there are no other parents then
            for y in node_values[Y]: # for each node values child takes 
                S = S + (cp[y][x]*lambda_value[y])  # sum of product of conditional probablity of Child given parent
        else:  # otherwise
            for y in node_values[Y]: # for each child with all values it takes
                s=0
                for variable in c: # for each other parent case
                    t = 1 # variable used to store intermediate values (initialized)
                    g = 0 # variable used to store intermediate values (initialized)
                    h = len(variable)
                    for g in range(0,h,2):
                        t = t*pi_msg[Y][str(variable[g])+str(variable[g+1])] # product of all pi messages for all combinations
                    s = s+ (t*cp[y][str(x)+str(variable)]) # sum of above product with all corresponding conditional probablities
                S = S + (lambda_value[y]*s) # Summation of product of above sum and Lambda value of Y for all its values

        lambda_msg[Y][x] = S # Lambda message value is above result
        t2 = 1
        
        for U in child[X]:  # For each child of X
            t2 = t2*lambda_msg[U][x]
        lambda_value[x] = t2  # update lambda value
        zee[x] = lambda_value[x]*pi_value[x] # zee[x] is a variable that holds the value of Probablity tilda as per pseudocode
    ALPHA=0   # This term is used to round off the probability to be less than or equal to one (marginalize)
    for k in zee.keys():
        ALPHA+=zee[k]
    for x in node_values[X]:
        if ALPHA==0:  # To avoid division by zero error
            updated_prob[x] = zee[x]
        else:
            updated_prob[x] = zee[x]/ALPHA # marginalize

    for Z in parents[X]:  # for each parent of X
        if Z not in evidence: # if not in evidence
            LAMBDA_MSG(X, Z) # send lambda message to its parents

    for U in child[X]: # for each child of X
        if U!=Y:   # except for Y 
            PI_MSG(X,U) # send PI message to its children

def Instances(parents_tilda):  # This is a function that returns other parents
        if len(parents_tilda)==0: # if empty list
            return []  #return empty other parents
        if len(parents_tilda)==1: 
            return node_values[parents_tilda[0]]
        
        cases = [] # list that holds updated other parents
        for i in node_values[parents_tilda[0]]:  # for each node values
            for j in node_values[parents_tilda[1]]: # its adjacent 
                string = str(i)+str(j)  # create all possible cases
                cases.append(string)
                      
        if len(parents_tilda)>2:  # for case where the length is greater than two
            i=2
            for i in range(len(parents_tilda)):
                cases = [str(x)+str(y) for x in cases for y in node_values[parents_tilda[i]]] # generate all cases
        final = list(set(cases)) # datatype conversion to list
        
        return final  

#"Function to pass PI message"####################################################################################

def PI_MSG(Z, X): # PI message from parent to its child
    for z in node_values[Z]: # for all values parent can take
        res=1
        for U in child[Z]: # for all its children
            if U !=X:  # Check for other child case
                res = res*lambda_msg[U][z] # update lambda message

        pi_msg[X][z] = pi_value[z]*res # update PI message
    c = Instances(parents[X]) # get all other possible cases
    zee = {} # zee[x] is a variable that holds the value of Probablity tilda as per pseudocode
    
    if X not in evidence: # for child that is not in evidence
        for x in node_values[X]: # for all values it takes
            S=0 # variable to store intermediate value
            for variable in c: # for all other cases
                t = 1 # variable used to store intermediate values (initialized)
                g = 0 # variable used to store intermediate values (initialized)
                h = len(variable) # to keep track of loop length based on other case length
                for g in range(0,h,2):
                    t = t*pi_msg[X][str(variable[g])+str(variable[g+1])] #Product of pi messages
                S = S+(cp[x][variable]*t) # summation of product of PI messages with conditional probablities

            pi_value[x] = S # update PI value based on above sum
            zee[x] = lambda_value[x]*pi_value[x]#Probability tilda mentioned in pseudocode
 
        ALPHA=0 #initialize factor to zero

        for k in zee.keys(): 
            ALPHA+=zee[k]

        for x in node_values[X]:
            if ALPHA==0:           # To avoid division by zero error
                updated_prob[x] = zee[x]
            else:
                updated_prob[x] = zee[x]/ALPHA # Marginalize

        for Y in child[X]: # for each child of X
            PI_MSG(X,Y) # send PI Message

    for x in node_values[X]: # for each values child takes
        if lambda_value[x] != 1: # Condition to check for V Structure
            for W in parents[X]: 
                if W !=Z and W not in evidence: # Parent other than Z and not in evidence then
                    LAMBDA_MSG(X,W) # Send Lambda message
            
##########################################################################################################################
###################################### INSTANTIATIONS for PART 1B OF FINAL PROJECT    #######################################
#########################################################################################################################
"""
Steps : 

1.Update all variables to initial conditions by calling "variable_set()" function.
2.Call "initialize_network()" function
3.Instantiate using "Instantiate_network()" by passing node and its value as arguements.
4.Print / Display required query node probablity by "updated_prob" variable with query node as its key.

"""

### "B0"

evidence,evidence_inst,updated_prob,lambda_value,pi_value,lambda_msg,pi_msg = variable_set()
initialize_network()
Instantiate_network('B','B0')
P_A1_Given_B0 =updated_prob['A1'] 



#### DISPLAY SOLUTIONS IN ORDER AS PER QUESTION FOR PART 1B #### 

print("\n")
print("P_A1_Given_B0 = ", + P_A1_Given_B0)


Directed Acyclic graph
for Node  A
	 we have structure as  []
				 ------>  A
						------>  ['C']
for Node  B
	 we have structure as  []
				 ------>  B
						------>  ['C']
for Node  C
	 we have structure as  ['A', 'B']
				 ------>  C
						------>  ['D', 'E']
for Node  D
	 we have structure as  ['C']
				 ------>  D
						------>  ['F', 'G']
for Node  E
	 we have structure as  ['C']
				 ------>  E
						------>  []
for Node  F
	 we have structure as  ['D']
				 ------>  F
						------>  []
for Node  G
	 we have structure as  ['D']
				 ------>  G
						------>  []


P_A1_Given_B0 =  0.7
